In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/PlantDiseaseFix

/content/drive/MyDrive/PlantDiseaseFix


In [ ]:
root_dir =  "/content/drive/MyDrive/PlantDiseaseFix"


In [ ]:
for i in os.listdir(root_dir):
  print(i)

Validation
Test
Train
path
plantdisease.tflite
model.tflite


In [ ]:
# Create path

train_path = os.path.join(root_dir, "Train")
test_path = os.path.join(root_dir, "Test")
val_path = os.path.join(root_dir, "Validation")


In [ ]:
image_size = 224
num_class = 3
batch_size = 32

In [ ]:
# Create Object ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Generate from directory
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(image_size, image_size),
        batch_size=batch_size)

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(image_size,image_size),
        batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory(
        val_path,
        target_size=(image_size,image_size),
        batch_size=batch_size)

Found 1331 images belonging to 3 classes.
Found 154 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [ ]:
efficient = tf.keras.applications.mobilenet.MobileNet(
    include_top=False, weights='imagenet',
    input_shape=(image_size, image_size, 3))

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
for layer in efficient.layers:
    layer.trainable = False

In [ ]:
# Add custom classification layers
model = tf.keras.Sequential([
    efficient,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model._name = 'mobilenet_plantdisease'

In [ ]:
import tensorflow as tf

# Define a custom callback
class AccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] >= 0.95:
            print("\nReached 95% accuracy. Stopping training.")
            self.model.stop_training = True

# Create an instance of the callback
accuracy_callback = AccuracyCallback()

# Compile and train the model with the callback
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
# Training
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=10,
          validation_data=validation_generator,
          validation_steps=len(validation_generator),
          callbacks=[accuracy_callback])

Epoch 1/10
42/42 [==============================] - 566s 13s/step - loss: 0.4568 - accuracy: 0.8355 - val_loss: 0.0440 - val_accuracy: 1.0000
Epoch 2/10
42/42 [==============================] - ETA: 0s - loss: 0.1201 - accuracy: 0.9624
Reached 95% accuracy. Stopping training.
42/42 [==============================] - 332s 8s/step - loss: 0.1201 - accuracy: 0.9624 - val_loss: 0.0541 - val_accuracy: 0.9667


In [ ]:
model.save('path/to/save/plantdisease.h5')

In [ ]:
from tensorflow import lite

In [ ]:
%cd "/content/drive/MyDrive/PlantDiseaseFix"

/content/drive/MyDrive/PlantDiseaseFix


In [ ]:
model = tf.keras.models.load_model('path/to/save/plantdisease.h5')


In [ ]:
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
# Save the TFLite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)